# Couplage et Coloration

## prérequis

### installer avec pip (sur la machine)

c'est pareil avec Windows, Linux, Macos

##### version de python :

>si vous avez la version python ou python3 il suffit d'ajoute le 3 en fonction de votre version<br>
>et heuresement python fonctionne avec pip et python3 fonctionne avec pip3 (normalement les versions<br>
>actuelles installées sont pip3 avec python3, python n'est plus utilisé donc pip non plus)<br>

#### installer dash_cytoscape avec 2 packages

source : https://manual.cytoscape.org/en/latest/Programmatic_Access_to_Cytoscape_Features_Scripting.html<br>
exemples : https://dash.plotly.com/cytoscape

```sh
pip3 install dash
pip3 install dash-cytoscape
```

#### installer pip3 :

```sh
python3 -m ensurepip --upgrade
```
enlever 3 dans python3 pour les version ultérieures<br>
si problèmes voir : https://pip.pypa.io/en/stable/installation/



rappel :

- planaire : un peut dessiner le graphe sans que deux lignes ne se croisent

In [3]:
# modules importants pour le(s) tp(s) :

# visualisation des couleurs et autres... (partie graphique)
from dash import Dash, html # type: ignore
import dash_cytoscape as cyto # type: ignore

# partie système
from time import time, sleep
import os, sys
from random import randint

# partie mathématique + structure
import numpy as np # type: ignore
from math import inf, sqrt, cos, sin, tan

In [15]:
# ON IMPORTE LES CLASSES DU TP PRECEDENT
class V:
    '''représente les informations des sommets (vertex)'''
    def __init__(self, name:str | int, weight=0):
        self.name = name
        self.weight = weight
class Vertex:
    """
    représente les sommets et leurs adjacences dans un graphe
    
    on crée la classe qui s'occupe de toutes les adjacences entre 
    les sommets (vertex/vertices) et leurs autres sommets reliés
    les arrêtes (edges) ne sont pas représentés car c'est la matrice d'adjacence qui s'occupe de ça
    qui est situé dans la classe Graph
    """
    id = 0 # identifiant du sommet 

    def __init__(self, name:str, neighbor:list[V]):
        self.name = name
        self.total_weight = sum([v.weight for v in neighbor]) # poids total du sommet
        self.neighbor = neighbor
        self.vertices_names = [v.name for v in self.neighbor]
        self.degree = len(neighbor)
        self.id = Vertex.id
        Vertex.id += 1

    def __str__(self):
        '''si on affiche Node (print(Node)) renverra ce qui suit'''
        res = f"{self.name} ["
        v_size = len(self.neighbor)
        for i, v in enumerate(self.neighbor):
            res += f"{v.name} w={v.weight}"
            if(i!=v_size-1):
                if v_size>1:
                    res+="," 
                res+=" "
        res+="]"
        return res
class Edge:
    '''représente une arrête entre deux sommets/vertex
    manière différente de représenter un graphe'''

    id = 0

    def __init__(self, vfrom:str | int, vto:str | int, weight: int | float):
        self.a = vfrom
        self.b = vto
        self.weight = weight
        self.id = Edge.id
        Edge.id += 1
    
    def __eq__(self, other:"Edge"): # equivalent to self == other
        '''permet d'utiliser l'objet dans un set (le rendre hashable)
        pour faire des comparaisons entre des ensembles (set) d'objet Edge'''
        # On considère que deux arêtes sont égales si leurs sommets et poids sont égaux
        return (self.a == other.a and self.b == other.b and self.weight == other.weight)
    
    def __ne__(self, edge:object): # equivalent de !=
        return not self.__eq__(edge)    

    def __hash__(self): # equivalent to set(self) == set(other)
        """permet d'utiliser l'objet dans un set (le rendre hashable)
        pour faire des comparaisons entre des ensembles (set) d'objet Edge"""
        # Calculer le hash en fonction des attributs de l'objet
        return hash((self.start, self.end, self.weight))

class Graph:
    """créée un graphe avec une liste de "Node" en paramètre et un titre
    @version 3.0.1"""

    def __init__(self, data:list[Vertex], title:str="default"): # OK
        '''@version 3.0.0'''
        self.title = title # titre du graphique si utilisé
        self.adj = data # liste d'adjacences
        self.edges = []
        self.verticesn = [] # liste des sommets (seulement les noms)
        self.weight = 0 # poids total du graphe    
        self.degree = 0 # pas encore calculé le degré du graphe
        self.is_complete = True
        for vx in self.adj:
            self.verticesn.append(vx.name)
            self.weight += vx.total_weight
            for v in vx.neighbor:
                self.edges.append(Edge(vx.name, v.name, v.weight))
                self.degree += 1
        self.create_matrix() # matrice des liens entre les noeuds (Edges) (matrice d'adjacences)
        self.port = 8054+randint(10,round(1e4))
        self.show_weights=True

    def create_matrix(self): # OK
        """créer la matrice d'ajdacence des points\n
        (fonctionne avec les graphes orienté également)
        @version {3.0.1}
        prochaine version supprimer l'initialisation de la matrice a False
        et ajouter un par un les True et False (reduction de complexité)
        """
        # si on trouve que m[i][j] != m[j][i] 
        # ou que les poids ne sont pas les mêmes c'est oriente
        # attention dans les Vertex les V peuvent ne pas être dans l'ordre
        self.matrix = [[False for _ in self.adj] for _ in self.adj]
        self.is_oriented=False
        for lin, vis in enumerate(self.adj):
            for v in vis.neighbor:
                v_jx_idx = self.verticesn.index(v.name)
                self.matrix[lin][v_jx_idx]=True
                if not self.is_oriented:
                    if vis.name not in self.adj[v_jx_idx].vertices_names:
                        self.is_oriented=True
                    else:
                        vi_xj_idx = self.adj[v_jx_idx].vertices_names.index(vis.name)
                        vi_ji = self.adj[v_jx_idx].neighbor[vi_xj_idx]
                        if v.weight != vi_ji.weight :
                            self.is_oriented=True
        iter = range(len(self.matrix))
        # matrice d'un graphe complet avec la liste des sommets actuel
        complete = [[self.matrix[i][j] if i == j else True for i in iter] for j in iter]
        if(self.matrix == complete):
            self.is_complete = True

    def __str__(self): # OK
        """si jamais on print un graph (print(Graph)) c'est executé ici
        affichage au plus simple du graphe avec des caractères
        @version 3.0.0"""
        res = ""
        for i, vis in enumerate(self.adj):
            res += f"{i}\t | {vis.name} ["
            v_size = len(vis.neighbor)
            for j in range(v_size):
                res += f"{vis.neighbor[j].name} w={vis.neighbor[j].weight}"
                if(j!=v_size-1):
                    if v_size>1:
                        res += "," 
                    res+=" "
            res += "]\n"
        return res
    
    def _sort_edges(self, edges:list[Edge])->list[Edge]: # OK complexite≈O(n+log(5n))
        """fonction privée a ne pas utiliser (en dehors de la classe)
        algorithme reccursif pour les problèmes de pronfondeur et de performances
        @version 3.0.0"""
        if len(edges)<2 :
            return edges
        else:
            pivot = edges[len(edges)//2].weight
            l, m, r = [],[],[] # mineurs, égal, majeurs 
            for e in edges:
                if(e.weight < pivot): l.append(e)
                    # si reccursion sur len(m) ce n'est jamais < 2
                    # et donc (boucle infini) dans certains cas                
                elif(e.weight == pivot): m.append(e) 
                else: r.append(e)
            return self._sort_edges(l)+m+self._sort_edges(r)

    def sort_by_weight(self): # OK
        """trie le graphe par poids croissants(asc)
        on trie chaque arrêtes du graphe
        @version 3.0.0
        """
        self.edges = self._sort_edges(self.edges)

    def show_edges(self):
        '''affiche les edges sous forme de liste de string
        @version 3.0.0'''
        print([e.a+e.b+" w="+str(e.weight) for e in self.edges])
    
    def render(self, layout_name="breadthfirst"):
        """effectue le rendu du graphe visuellement
        @version 3.1.4"""
        unique = f' {time()%1e4:.5}'
        app = Dash(self.title+unique)
        allow_arrows = "linear" # ce style n'autorise pas les flèches
        if self.is_oriented:
            allow_arrows = "bezier" # ce style oui
        custom_style = {
            'width': '100%', 
            'height': '500px',
            "border": "3px white solid",
            "border-radius":"5px",
            "background-color":"#666666",
            "title" : {"background-color":"white"}
        }
        my_styles_sheet = [{
                'selector': 'node',
                'style': {
                    'background-color': '#222222', 
                    'color': 'white',
                    'label': 'data(label)',
                    'font-size': '16px',
                    'text-valign': 'center', 
                    'text-halign': 'center' 
                }
            },
            {
                'selector': 'edge',
                'style': {
                    'width': 2,
                    'target-arrow-shape': "vee",
                    "target-arrow-color": "#4a7cf2",
                    'arrow-scale': 2,
                    'curve-style': allow_arrows
                }
            },

        ]
        if self.show_weights:
            my_styles_sheet.append({
                'selector': 'edge',
                'style': {
                    'label': 'data(weight)',
                    "color": 'white'
                }
            })
        elems = [] # éléments à afficher (formattés)
        for node in self.adj:
            elems.append({'data': {"id":node.name, "label":node.name}})
        # add edges
        for edge in self.edges:
            elems.append({
                'data': {
                    'source': edge.a, 
                    'target': edge.b, 
                    'weight': edge.weight
                }
            })
        app.layout = html.Div([
            cyto.Cytoscape(
                id='cytoscape'+unique,
                elements=elems,
                layout={'name': layout_name},
                style=custom_style,
                stylesheet=my_styles_sheet
            )
        ])
        print('\nrendu graphique : ')
        print(f"\tégalement ouvert sur la page web : \"localhost:{self.port}\"")
        print(f"\topened too at the web page : \"localhost:{self.port}\"")
        app.run_server(debug=True,port=self.port)

    def resume(self):
        '''crée un résumé du graphe
        @version 3.0.0'''
        print("quelques informations sur le graphe : \n")
        print(f"\tdegré: {self.degree}")
        print(f"\tpoids: {self.weight}")
        print(f"\tcomplet : {self.is_complete} (def: si tous les sommets sont reliés)")
        print(f"\tplanaire: voir si les arrêtes se croisent ou non")
        print(f"\t\tnecessite surement de déplacer\n\t\tles noeuds (sur la partie graphique)")

    def coloration(self):
        """
        Réalise une coloration des sommets d'un graphe donné.
        renvoie un dict: Un dictionnaire associant chaque sommet à une couleur.
        """
        # Étape 1 : Construire les degrés des sommets
        degres = {vertex.name: len(vertex.neighbor) for vertex in self.graph.adj}
        # Étape 2 : Trier les sommets par ordre décroissant de degré
        sommets_tries = sorted(self.graph.adj, key=lambda v: len(v.neighbor), reverse=True)
        # Étape 3 : Initialiser les couleurs
        coloration = {vertex.name: None for vertex in self.graph.adj}
        couleur_max = 0
        # Étape 4 : Attribuer les couleurs
        for vertex in sommets_tries:
            # Collecter les couleurs déjà utilisées par les voisins
            couleurs_voisines = {
                coloration[neighbor.name]
                for neighbor in vertex.neighbor
                if coloration[neighbor.name] is not None
            }
            # Trouver la première couleur non utilisée
            couleur = 1
            while couleur in couleurs_voisines:
                couleur += 1
            # Assigner la couleur au sommet
            coloration[vertex.name] = couleur
            couleur_max = max(couleur_max, couleur)

        print(f"Nombre de couleurs utilisées : {couleur_max}")
        return coloration

        



## données
```
1: 2, 3
2: 1, 4
3: 1, 4, 6
4: 2, 3, 5, 7
5: 4, 7, 9
6: 3, 7, 10
7: 4, 5, 6, 8, 10
8: 7, 9, 10
9: 5, 8, 10
10: 6, 7, 8, 9
```

In [17]:
# rappel format du Vertex : 
# Vertex(nomActuel, [V("nomLié1", poid1),V("nomLié2", poid2),etc...])
data = [
    Vertex("1",[V("2"),V("3")]),
    Vertex("2",[V("1"),V("4")]),
    Vertex("3",[V("1"),V("4"),V("6")]),
    Vertex("4",[V("2"),V("3"),V("5"),V("7")]),
    Vertex("5",[V("4"),V("7"),V("9")]),
    Vertex("6",[V("3"),V("7"),V("10")]),
    Vertex("7",[V("4"),V("5"),V("6"),V("8"),V("10")]),
    Vertex("8",[V("7"),V("9"),V("10")]),
    Vertex("9",[V("5"),V("8"),V("10")]),
    Vertex("10",[V("6"),V("7"),V("8"),V("9")]),
]
graphe = Graph(data,"graphe a colorer")
graphe.show_weights=False
graphe.render("grid")
print(graphe.coloration())


rendu graphique : 
	également ouvert sur la page web : "localhost:9449"
	opened too at the web page : "localhost:9449"


AttributeError: 'Graph' object has no attribute 'graph'

### question
1. Une assemblée est formée de personnes parlant plusieurs langues<br>
différentes (voir la liste des données au dessus). On veut former des binômes de<br>
personnes qui pourront dialoguer entre elles. Comment maximiser le<br>
nombre de binômes ?

In [1]:
# code

2. Une enseignante essaie de former le plus de groupes de deux selon les<br>
affinités des étudiants. Elle a dressé le tableau d’incompatibilités ci-après,<br>
où une croix indique que deux personnes sont incompatibles. Combien de
couples pourra-t-elle former au maximum?

(problème inverse)

### traitement

vérification des résultats et conclusion

code original (R) : 

```R
sommets<-c(1,3,5,6,2,4)
arretes<-matrix(c(1,3,1,2,6,2,5,4),nrow=2,ncol=4)
coloration <- function(sommets,arets){
  taille=length(sommets)
  M<-matrix(rep(0,taille^2,taille,taille))
  print(length(arets))  
  for (i in 1:length(arets)/2){
    M[arets[i,1],arets[i,2]]=1
    M[arets[i,2],arets[i,1]]=1
    M[i,i]=1
  }
  degres<-rowSums(M)
  ordre_degres <- sort(degres,index.return=TRUE)
  sommets<-sommeets[ordre_degres$ix]
  i<-length(sommets)
  sommet_choisi<-c(rep(0,i))
  A = FALSE
  while(A==FALSE){
    new_color = c(rep(0),i)
    if(sommet_choisi[i]==0){
      sommet_choisi[i]=1
      new_color[sommets[i]] = 1
      a<-wich(M[sommets[i],]==0)
      if(length(a)>0){
        M[sommets[i],a[1]]=i
      } else {
        A=TRUE
      }
    }  
  }
}
coloration(arretes,sommets)

```

In [36]:
os.system(f"netstat -A{'n'*50}a | grep LISTEN")

ff0ba53e6f395b0d        0 tcp4       0      0  127.0.0.1.9449     *.*                LISTEN     
 b5e552e2a622c9b        0 tcp4       0      0  127.0.0.1.9319     *.*                LISTEN     
b1b09b28d18d1242        0 tcp4       0      0  127.0.0.1.8730     *.*                LISTEN     
4a63a16ad9e306d3        0 tcp4       0      0  127.0.0.1.12118    *.*                LISTEN     
7e2c7cfdb5f2cf79        0 tcp4       0      0  127.0.0.1.14395    *.*                LISTEN     
912fb9e511b59542        0 tcp4       0      0  127.0.0.1.16073    *.*                LISTEN     
a2868bed74190454        0 tcp4       0      0  127.0.0.1.8480     *.*                LISTEN     
eb0a3f68f78d1f64        0 tcp4       0      0  127.0.0.1.14884    *.*                LISTEN     
a1a855727bd67d43        0 tcp4       0      0  127.0.0.1.16424    *.*                LISTEN     
a8a7b5077469c7bf        0 tcp4       0      0  127.0.0.1.9005     *.*                LISTEN     
80d5d0b8b2b739cd        0 tcp4

0

In [37]:
os.system("netstat -n")

Active Internet connections
Proto Recv-Q Send-Q  Local Address          Foreign Address        (state)    
tcp4       0      0  192.168.0.40.51048     140.82.121.5.443       ESTABLISHED
tcp4       0      0  192.168.0.40.50947     140.82.113.26.443      ESTABLISHED
tcp4       0      0  192.168.0.40.50813     140.82.112.26.443      ESTABLISHED
tcp4       0      0  192.168.0.40.50764     20.42.65.88.443        ESTABLISHED
tcp4       0      0  127.0.0.1.9008         127.0.0.1.49474        ESTABLISHED
tcp4       0      0  127.0.0.1.49474        127.0.0.1.9008         ESTABLISHED
tcp4       0      0  127.0.0.1.9006         127.0.0.1.49473        ESTABLISHED
tcp4       0      0  127.0.0.1.49473        127.0.0.1.9006         ESTABLISHED
tcp4       0      0  127.0.0.1.9007         127.0.0.1.49472        ESTABLISHED
tcp4       0      0  127.0.0.1.49472        127.0.0.1.9007         ESTABLISHED
tcp4       0      0  127.0.0.1.9009         127.0.0.1.49471        ESTABLISHED
tcp4       0      0  127

0